In [1]:
import re
from collections import defaultdict, Counter
from typing import List, Tuple
import math

In [2]:
class NGramModel:
    def __init__(self, n: int):
        self.n = n
        self.ngram_counts = defaultdict(Counter)
        self.vocab = set()
        self.total_ngrams = 0

    def _tokenize(self, text: str) -> List[str]:
        """Convierte texto en una lista de palabras con tokens de inicio/fin."""
        text = re.sub(r'[^\w\s]', '', text.lower())  # limpia puntuación y pasa a minúsculas
        tokens = text.split()
        tokens = ['<s>'] * (self.n - 1) + tokens + ['</s>']
        return tokens

    def _generate_ngrams(self, tokens: List[str]) -> List:
        """Genera pares (contexto, palabra objetivo)"""
        ngrams = []
        for i in range(len(tokens) - self.n + 1):
            context = tuple(tokens[i:i + self.n - 1])
            target = tokens[i + self.n - 1]
            ngrams.append((context, target))
        return ngrams

    def train(self, texts: List[str]):
        """Entrena el modelo con una lista de párrafos o frases."""
        for text in texts:
            tokens = self._tokenize(text)
            ngrams = self._generate_ngrams(tokens)
            for context, target in ngrams:
                self.ngram_counts[context][target] += 1
                self.vocab.add(target)
                self.total_ngrams += 1

        # Cálculo para Good-Turing
        self.count_of_counts = Counter()
        for context in self.ngram_counts:
            for word in self.ngram_counts[context]:
                count = self.ngram_counts[context][word]
                self.count_of_counts[count] += 1

    def predict_next(self, context: Tuple[str]) -> List[Tuple[str, float]]:
        """Predice la próxima palabra más probable dado un contexto."""
        if len(context) != self.n - 1:
            raise ValueError(f"El contexto debe tener longitud {self.n - 1}")

        target_counts = self.ngram_counts.get(context, {})
        total = sum(target_counts.values())

        if total == 0:
            return []

        return sorted(
            [(word, count / total) for word, count in target_counts.items()],
            key=lambda x: x[1],
            reverse=True
        )
    
    def good_turing_probability(self, context: Tuple[str, ...], word: str) -> float:
        r = self.ngram_counts[context][word]
        Nr = self.count_of_counts[r]
        Nr_plus_1 = self.count_of_counts.get(r + 1, 0)

        # Good-Turing smoothing only applies when we have observed data
        if Nr > 0 and Nr_plus_1 > 0:
            r_star = (r + 1) * (Nr_plus_1 / Nr)
        else:
            r_star = 1e-10  # Asignamos una probabilidad muy baja si no hay datos

        total_context = sum(self.ngram_counts[context].values())
        normalizer = total_context if total_context > 0 else 1

        return r_star / normalizer

    def probability(self, context: Tuple[str, ...], word: str, k: float = 1.0, smoothing: str = "laplace") -> float:
        """Devuelve la probabilidad de una palabra dado el contexto, usando Laplace o Good-Turing."""
        if len(context) != self.n - 1:
            raise ValueError(f"El contexto debe tener longitud {self.n - 1}")

        if smoothing == "laplace":
            V = len(self.vocab)
            count = self.ngram_counts[context][word] + k
            total = sum(self.ngram_counts[context].values()) + k * V
            return count / total
        elif smoothing == "good_turing":
            return self.good_turing_probability(context, word)
        else:
            raise ValueError("Smoothing no reconocido. Usa 'laplace' o 'good_turing'.")
    
    def entropy(self, sentence: str, k: float = 1.0, smoothing: str="laplace") -> Tuple[float, float]:
        tokens = self._tokenize(sentence)
        ngrams = self._generate_ngrams(tokens)

        entropy = 0
        for context, word in ngrams:
            prob = self.probability(context, word, k=k, smoothing=smoothing)
            entropy += -math.log2(prob)
        
        entropy_per_word = entropy / len(ngrams)
        perplexity = 2 ** entropy_per_word
        
        return entropy_per_word, perplexity

In [3]:
# Creamos un modelo de bigramas
modelo = NGramModel(n=2)

# Entrenamos con texto
corpus = [    
    "El procesamiento de lenguaje natural es una rama de la inteligencia artificial que permite a las máquinas comprender y generar lenguaje humano. Esta tecnología se encuentra en aplicaciones cotidianas como asistentes virtuales, traductores automáticos y sistemas de recomendación. Su objetivo es reducir la brecha entre la forma en que las personas se comunican y cómo las computadoras procesan información.",
    "Los modelos estadísticos como los N-gramas son fundamentales para tareas de PLN. Un modelo N-gram aprende las secuencias de palabras más comunes en un texto y asigna probabilidades a las siguientes palabras, basado en el contexto. Aunque es un enfoque simple comparado con modelos modernos, sigue siendo útil por su facilidad de implementación y análisis.",
    "A pesar de sus ventajas, los N-gramas enfrentan el problema de datos escasos, ya que es difícil cubrir todas las combinaciones posibles de palabras. Para solucionar esto, se aplican técnicas como el smoothing, que ajustan las probabilidades para evitar ceros. Esto mejora la robustez del modelo al trabajar con nuevos textos o frases no vistas durante el entrenamiento."
    ]
modelo.train(corpus)

contexto = ("encuentra",)
print(modelo.predict_next(contexto)) 
  

[('en', 1.0)]


Si la palabra es "encuentra" la palabra más probable después es "en".

Esto con una probabilidad de 1 dentro de lo que sabe.

## Probabilidad de una palabra específica

In [4]:
# Obtener probabilidad específica
print(modelo.probability(("encuentra",), "en"))

0.01694915254237288


## Probabilidad con Smoothing Laplaciano

In [5]:
# Obtener probabilidad específica
print(modelo.probability(("encuentra",), "dulce", smoothing="laplace", k=1.0))

0.00847457627118644


## Probabilidad con Smoothing Good-Turing

In [6]:
# Obtener probabilidad específica
print(modelo.probability(("encuentra",), "dulce", smoothing="good_turing"))

1e-10


La probabilidad usando smoothing  de que "en" esta después de "encuentra" es de 0.017.

In [7]:
test_sentence = "se esta probando un ngramas con el problema de datos escasos"
H, PP = modelo.entropy(test_sentence)
print(f"Entropia: {H}")
print(f"Perplejidad: {PP}")

Entropia: 6.568271790961251
Perplejidad: 94.89576446460957


Entropía = 6.568: en promedio el modelo necesita mucha información (6.57 bits) para adivinar bien la siguiente palabra.

Perplejidad = 94.896: en promedio hay 94 palabras posibles para cada contexto, esto es muy alto.

Entonces por la alta entropía (alta incertidumbre)  y también la alta perpelejidad (alta indecisión) podemos decir que por ahora el modelo no es muy bueno. Esto es debido a que por ahora se ha entrenado con un corpus muy limitado. 

No obstante, gracias al smoothing el modelo no colapsa ante combinaciones que tienen poca o nula frecuencia.

# ACTIVIDAD

1. el procesamiento de lenguaje	

In [8]:
contexto = ("lenguaje",)
print(modelo.predict_next(contexto)) 

[('natural', 0.5), ('humano', 0.5)]


2. el problema de datos

In [9]:
contexto = ("datos",)
print(modelo.predict_next(contexto)) 

[('escasos', 1.0)]


3. 	los modelos estadísticos

In [10]:
contexto = ("estadísticos",)
print(modelo.predict_next(contexto)) 

[('como', 1.0)]


4. 	la inteligencia

In [11]:
contexto = ("inteligencia",)
print(modelo.predict_next(contexto)) 

[('artificial', 1.0)]


5. los ngramas

In [12]:
contexto = ("ngramas",)
print(modelo.predict_next(contexto)) 

[('son', 0.5), ('enfrentan', 0.5)]
